# Colab Inference (Text to Video + Screenshots)

Generates a video from a prompt using your base model + LoRA weights. You can specify when to show screenshots either via prompt tags or a beats JSON.

## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## Clone repo (video-finetune)

In [ ]:
%%bash
set -euo pipefail
if [ -d /content/video-finetune ]; then
  echo 'Repo exists, pulling latest...'
  cd /content/video-finetune && git pull --ff-only || true
else
  git clone https://github.com/borjagomez/video-finetune.git /content/video-finetune
fi
ls -la /content/video-finetune | head -n 20


In [ ]:
import os, sys
os.chdir('/content/video-finetune')
print('CWD:', os.getcwd())


## Paths & Inputs

In [ ]:
from pathlib import Path
MOUNT = Path('/content/drive') / 'My Drive'
MODEL_DIR = MOUNT / 'HunyuanVideo-diffusers'
LORA_WEIGHTS = MOUNT / 'outputs' / 'lora' / 'lora_weights.pt'
SCREENSHOTS_DIR = MOUNT / 'screenshots'  # put reference screenshots here
BEATS_JSON = MOUNT / 'outputs' / 'inference' / 'beats.json'
OUT_MP4 = MOUNT / 'outputs' / 'inference' / 'generated.mp4'
# Story prompt with inline SHOT tags (timed inserts)
PROMPT = """A clear, friendly walkthrough of the product dashboard showing key sections.
[SHOT at=2.0 file=login.png dur=2.0]
Then highlight the dashboard charts and filters.
[SHOT time=6.5 screenshot=/content/drive/My Drive/screenshots/dashboard.png duration=2s]
"""
print('Model dir:', MODEL_DIR)
print('LoRA weights:', LORA_WEIGHTS)
print('Screenshots dir:', SCREENSHOTS_DIR)
print('Beats JSON:', BEATS_JSON)
print('Output path:', OUT_MP4)


## Option B: Beats JSON (alternative)
If you already have a beats JSON, place it at the path below; otherwise rely on prompt tags.

In [ ]:
from pathlib import Path
print('Beats JSON exists:', BEATS_JSON.exists(), '->', BEATS_JSON)


## Install deps

## System deps (ffmpeg)

In [ ]:
%%bash
set -euo pipefail
apt-get -y update >/dev/null 2>&1 || true
apt-get -y install ffmpeg >/dev/null 2>&1 || true
ffmpeg -version | head -n1


In [ ]:
%%bash
set -euo pipefail
python -V
pip install -U pip setuptools wheel >/dev/null 2>&1 || true
if [ -f requirements.txt ]; then
  pip install -r requirements.txt
else
  pip install 'diffusers>=0.30' 'transformers>=4.43' 'accelerate>=0.30' safetensors Pillow imageio-ffmpeg
fi


## Sanity checks

In [ ]:
from pathlib import Path
assert Path('tools/generate_video.py').exists(), 'Run this notebook from the repo root.'
print('Model folder exists:', MODEL_DIR.exists())
print('LoRA file exists:', LORA_WEIGHTS.exists())
print('Will glob screenshots from:', str(SCREENSHOTS_DIR / '*'))


## Generate video

## Memory saver (optional)

In [ ]:
import os, gc, torch
os.environ['PYTORCH_CUDA_ALLOC_CONF']='expandable_segments:True'
print('PYTORCH_CUDA_ALLOC_CONF=', os.environ['PYTORCH_CUDA_ALLOC_CONF'])
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()


In [ ]:
import subprocess
from pathlib import Path as _P
OUT_MP4.parent.mkdir(parents=True, exist_ok=True)
args = [
  'python','tools/generate_video.py',
  '--model_dir', str(MODEL_DIR),
  '--lora_weights', str(LORA_WEIGHTS),
  '--prompt', PROMPT,
  '--out', str(OUT_MP4),
  '--width','256','--height','256',
  '--num_frames','24','--fps','24','--steps','20','--guidance','1.0',
  '--screenshot_duration','1.5',
  '--beats_from_prompt',
  '--shots_base_dir', str(SCREENSHOTS_DIR),
]
# If a beats JSON exists, prefer it over prompt tags
if _P(BEATS_JSON).exists():
    args += ['--beats', str(BEATS_JSON)]
print('Running:', ' '.join(args))
try:
    res = subprocess.run(args, check=True, capture_output=True, text=True)
    print(res.stdout)
except subprocess.CalledProcessError as e:
    print('Command failed with exit code', e.returncode)
    print('--- STDOUT ---')
    print(e.stdout)
    print('--- STDERR ---')
    print(e.stderr)
    raise


## Output
The generated video is at the `Output path` above. If beats were provided or parsed from the prompt, a `_with_beats.mp4` version includes the integrated screenshots.